# word2vec: How To Implement word2vec

### Explore Pre-trained Embeddings

Some other options:
- `glove-twitter-{25/50/100/200}`
- `glove-wiki-gigaword-{50/200/300}`
- `word2vec-google-news-300`
- `word2vec-ruscorpora-news-300`

In [ ]:
# Install gensim

In [ ]:
# Load pretrained word vectors using gensim
import gensim.downloader as api

wiki_embeddings = api.load('glove-wiki-gigaword-100')

In [ ]:
# Explore the word vector for "king"

In [ ]:
# Find the words most similar to king based on the trained word vectors

### Train Our Own Model

In [ ]:
# Read in the data and clean up column names
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('../../../data/spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]
messages.head()

In [ ]:
# Clean data using the built in cleaner in gensim
messages['text_clean'] = messages['text'].apply()
messages.head()

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

In [ ]:
# Train the word2vec model
w2v_model = gensim.models.Word2Vec()

In [ ]:
# Explore the word vector for "king" base on our trained model

In [ ]:
# Find the most similar words to "king" based on word vectors from our trained model